In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = '../train/'
    TRAIN_SUB_DIR = '../subsample/'
    TRAIN_GAN = '../../image_gan/'
    TEST_DIR = '../../test/'

    # use this for full dataset
    train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
    test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
    train_images = train_images_gan
    
    train_images.sort(key=natural_keys)
    test_images.sort(key=natural_keys)

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []
    testImage = []
    testLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
      # loop over the input images
    for (i, imagePath) in enumerate(test_images):
        # extract the class label
        # our images were named as labels.image_number.format
        # get the labels from the name of the images by extract the string before "."
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # extract CNN features in the image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        testImage.append(img)
        testLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    testImage = np.array(testImage,dtype = float32)
    testLabels = np.array(testLabels)
    print (trainImage.shape)
    
    trainImage = trainImage.astype(np.float32) / 255.0
    testImage = testImage.astype(np.float32) / 255.0
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    testLabels = le.transform(testLabels) 
    
    return trainImage, trainLabels, testImage, testLabels

In [3]:
trainImage, trainLabels, testImage, testLabels = load_flower_data()

nb_classes = 4

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
testLabels = keras.utils.to_categorical(testLabels, nb_classes)
print (testLabels)
print (testLabels.shape)

np.save('../trainImage.npy', trainImage)
np.save('../trainLabels.npy', trainLabels)
np.save('../testImage.npy', testImage)
np.save('../testLabels.npy', testLabels)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage matrix: {:.2f}MB".format(
    (testImage.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels matrix: {:.4f}MB".format(
    (testLabels.nbytes) / (1024 * 1000.0)))


[INFO] handling images...
[INFO] processed 1000/4000
[INFO] processed 2000/4000
[INFO] processed 3000/4000
[INFO] processed 4000/4000
[INFO] processed 312/312
(4000, 128, 128, 3)
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
(312, 4)
[INFO] trainImage matrix: 768.00MB
[INFO] trainLabels matrix: 0.1250MB
[INFO] testImage matrix: 59.90MB
[INFO] testLabels matrix: 0.0097MB


In [4]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 4
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]    
        # number of dense filters to use  
        self.nb_Dense = [128] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # train   
        self.global_step = tf.Variable(0, trainable=False)
        self.initial_lr = 0.001 #初始学习率
        self.decay_steps = 10
        self.decay_rate = 0.9
        self.lr = tf.train.exponential_decay( self.initial_lr,
                                     global_step=self.global_step,
                                     decay_steps=self.decay_steps,
                                     decay_rate=self.decay_rate)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # name for checkpoint
        self.model_name = 'CNN_C%d_D%d_Kernel(%d,%d)_%d_lrdecay0.0001' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))    
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y= np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y= np.load('../testLabels.npy')
        
        print ("Training:",self.train_x.shape[0])
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 

            #输入,卷积核为3*3 输出维度为32
            net = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1'               # 命名用于获取变量
                                    )
            print("CNN:",net.get_shape())

            if self.batch_normalization_control:
                net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            net = tf.layers.max_pooling2d(inputs = net,
                                          pool_size = self.pool_size,
                                          strides = (2, 2),
                                          padding='same',
                                          name='pool_conv_' + str(1)
                                         )
            print("CNN:",net.get_shape())
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )

                print("CNN:",net.get_shape())

            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
            
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'_train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status            
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, 
                                                                                    self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                rand_index = np.random.choice(len(self.train_x), size=batch_size)
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                   self.y: batch_y_tes,
                                                                   self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
            
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
                
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append(test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '_train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '_train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
    
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")
        
    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')
                          
        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [5]:
gan_type = 'CNN'
dataset = '4_Flowers'
epoch = 80
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
    #CNN.test(epoch)
        
sess.close()
        

# lrdecay
# Avg per epoch ptime: 7.42, total 80 epochs ptime: 656.39
#  [*] Training finished!
#  [*] Best Epoch:  52 , Accuracy:  0.7566666603088379
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001-52
#  [*] Finished testing Best Epoch: 52 , accuracy:  0.7566666603088379 

Training: 4000
CNN:x (100, 128, 128, 3)
CNN: (100, 128, 128, 32)
CNN: (100, 64, 64, 32)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 4)
------------------------
CNN:x (100, 128, 128, 3)
CNN: (100, 128, 128, 32)
CNN: (100, 64, 64, 32)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 4)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/beta:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/gamma:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_2/kernel:0 (float32_ref 3x3x32x64) [184

Epoch: [ 3] [  24/  40] time: 23.0073, loss: 0.10127540
Epoch: [ 3] [  25/  40] time: 23.1623, loss: 0.04453087
Epoch: [ 3] [  26/  40] time: 23.3269, loss: 0.12516019
Epoch: [ 3] [  27/  40] time: 23.4830, loss: 0.09885577
Epoch: [ 3] [  28/  40] time: 23.6440, loss: 0.06477593
Epoch: [ 3] [  29/  40] time: 23.8042, loss: 0.10865870
Epoch: [ 3] [  30/  40] time: 23.9739, loss: 0.08953790
Epoch: [ 3] [  31/  40] time: 24.1287, loss: 0.03657631
Epoch: [ 3] [  32/  40] time: 24.2989, loss: 0.11377960
Epoch: [ 3] [  33/  40] time: 24.4535, loss: 0.05032029
Epoch: [ 3] [  34/  40] time: 24.6147, loss: 0.20485201
Epoch: [ 3] [  35/  40] time: 24.7686, loss: 0.04011295
Epoch: [ 3] [  36/  40] time: 24.9298, loss: 0.11488200
Epoch: [ 3] [  37/  40] time: 25.0850, loss: 0.03268770
Epoch: [ 3] [  38/  40] time: 25.2513, loss: 0.15272111
Epoch: [ 3] [  39/  40] time: 25.4091, loss: 0.04168971
[3/80] - ptime: 7.2712 loss: 0.09239663 acc: 0.27000 lr: 0.00096889
Epoch: [ 4] [   0/  40] time: 27.335

Epoch: [ 7] [   6/  40] time: 52.5757, loss: 0.02611587
Epoch: [ 7] [   7/  40] time: 52.7400, loss: 0.02153775
Epoch: [ 7] [   8/  40] time: 52.9035, loss: 0.00336585
Epoch: [ 7] [   9/  40] time: 53.0560, loss: 0.07355763
Epoch: [ 7] [  10/  40] time: 53.2188, loss: 0.02601997
Epoch: [ 7] [  11/  40] time: 53.3715, loss: 0.02383443
Epoch: [ 7] [  12/  40] time: 53.5371, loss: 0.03040129
Epoch: [ 7] [  13/  40] time: 53.7035, loss: 0.01844444
Epoch: [ 7] [  14/  40] time: 53.8701, loss: 0.01102761
Epoch: [ 7] [  15/  40] time: 54.0335, loss: 0.05651398
Epoch: [ 7] [  16/  40] time: 54.1982, loss: 0.04570487
Epoch: [ 7] [  17/  40] time: 54.3498, loss: 0.02903097
Epoch: [ 7] [  18/  40] time: 54.5217, loss: 0.04309350
Epoch: [ 7] [  19/  40] time: 54.6776, loss: 0.02038579
Epoch: [ 7] [  20/  40] time: 54.8377, loss: 0.03251067
Epoch: [ 7] [  21/  40] time: 54.9910, loss: 0.02023860
Epoch: [ 7] [  22/  40] time: 55.1531, loss: 0.02017171
Epoch: [ 7] [  23/  40] time: 55.3154, loss: 0.0

Epoch: [10] [  30/  40] time: 80.8306, loss: 0.05667293
Epoch: [10] [  31/  40] time: 80.9853, loss: 0.07606010
Epoch: [10] [  32/  40] time: 81.1472, loss: 0.01366362
Epoch: [10] [  33/  40] time: 81.3158, loss: 0.01294427
Epoch: [10] [  34/  40] time: 81.4763, loss: 0.01780846
Epoch: [10] [  35/  40] time: 81.6336, loss: 0.02415243
Epoch: [10] [  36/  40] time: 81.7946, loss: 0.00559445
Epoch: [10] [  37/  40] time: 81.9510, loss: 0.12906933
Epoch: [10] [  38/  40] time: 82.1152, loss: 0.03219521
Epoch: [10] [  39/  40] time: 82.2761, loss: 0.02433752
[10/80] - ptime: 7.3394 loss: 0.03486371 acc: 0.54000 lr: 0.00090000
Epoch: [11] [   0/  40] time: 84.0860, loss: 0.10393917
Epoch: [11] [   1/  40] time: 84.2426, loss: 0.04594719
Epoch: [11] [   2/  40] time: 84.4034, loss: 0.02026123
Epoch: [11] [   3/  40] time: 84.5551, loss: 0.07732952
Epoch: [11] [   4/  40] time: 84.7167, loss: 0.00443141
Epoch: [11] [   5/  40] time: 84.8721, loss: 0.04546740
Epoch: [11] [   6/  40] time: 85.03

Epoch: [14] [  12/  40] time: 110.4334, loss: 0.01013692
Epoch: [14] [  13/  40] time: 110.5938, loss: 0.03066243
Epoch: [14] [  14/  40] time: 110.7563, loss: 0.00561054
Epoch: [14] [  15/  40] time: 110.9171, loss: 0.00135035
Epoch: [14] [  16/  40] time: 111.0835, loss: 0.00958203
Epoch: [14] [  17/  40] time: 111.2385, loss: 0.00835301
Epoch: [14] [  18/  40] time: 111.4117, loss: 0.03624703
Epoch: [14] [  19/  40] time: 111.5676, loss: 0.02250936
Epoch: [14] [  20/  40] time: 111.7285, loss: 0.01102681
Epoch: [14] [  21/  40] time: 111.8816, loss: 0.01217277
Epoch: [14] [  22/  40] time: 112.0406, loss: 0.02590982
Epoch: [14] [  23/  40] time: 112.1967, loss: 0.00579549
Epoch: [14] [  24/  40] time: 112.3649, loss: 0.01816722
Epoch: [14] [  25/  40] time: 112.5273, loss: 0.00729958
Epoch: [14] [  26/  40] time: 112.6916, loss: 0.00257921
Epoch: [14] [  27/  40] time: 112.8501, loss: 0.01399746
Epoch: [14] [  28/  40] time: 113.0137, loss: 0.03815986
Epoch: [14] [  29/  40] time: 1

...

Epoch: [63] [  26/  40] time: 513.9635, loss: 0.00450825
Epoch: [63] [  27/  40] time: 514.1212, loss: 0.00182636
Epoch: [63] [  28/  40] time: 514.2875, loss: 0.00047891
Epoch: [63] [  29/  40] time: 514.4404, loss: 0.00054745
Epoch: [63] [  30/  40] time: 514.6051, loss: 0.00028074
Epoch: [63] [  31/  40] time: 514.7594, loss: 0.02135084
Epoch: [63] [  32/  40] time: 514.9260, loss: 0.01662589
Epoch: [63] [  33/  40] time: 515.0918, loss: 0.00029868
Epoch: [63] [  34/  40] time: 515.2554, loss: 0.08773075
Epoch: [63] [  35/  40] time: 515.4082, loss: 0.00082210
Epoch: [63] [  36/  40] time: 515.5697, loss: 0.00549467
Epoch: [63] [  37/  40] time: 515.7231, loss: 0.04475723
Epoch: [63] [  38/  40] time: 515.8898, loss: 0.00570577
Epoch: [63] [  39/  40] time: 516.0492, loss: 0.01433351
[63/80] - ptime: 7.4899 loss: 0.00593358 acc: 0.73333 lr: 0.00051491
Epoch: [64] [   0/  40] time: 517.9965, loss: 0.00128510
Epoch: [64] [   1/  40] time: 518.1526, loss: 0.01770134
Epoch: [64] [   2/ 

Epoch: [67] [   6/  40] time: 543.3930, loss: 0.00004118
Epoch: [67] [   7/  40] time: 543.5621, loss: 0.00016849
Epoch: [67] [   8/  40] time: 543.7266, loss: 0.00017785
Epoch: [67] [   9/  40] time: 543.8819, loss: 0.00001039
Epoch: [67] [  10/  40] time: 544.0448, loss: 0.00049222
Epoch: [67] [  11/  40] time: 544.2045, loss: 0.00021965
Epoch: [67] [  12/  40] time: 544.3758, loss: 0.00005504
Epoch: [67] [  13/  40] time: 544.5350, loss: 0.00011136
Epoch: [67] [  14/  40] time: 544.6953, loss: 0.00036891
Epoch: [67] [  15/  40] time: 544.8551, loss: 0.00330240
Epoch: [67] [  16/  40] time: 545.0180, loss: 0.00016196
Epoch: [67] [  17/  40] time: 545.1718, loss: 0.00006823
Epoch: [67] [  18/  40] time: 545.3372, loss: 0.00075459
Epoch: [67] [  19/  40] time: 545.4967, loss: 0.00001435
Epoch: [67] [  20/  40] time: 545.6572, loss: 0.00556583
Epoch: [67] [  21/  40] time: 545.8137, loss: 0.00002595
Epoch: [67] [  22/  40] time: 545.9761, loss: 0.00145493
Epoch: [67] [  23/  40] time: 5

Epoch: [70] [  28/  40] time: 571.4816, loss: 0.00003168
Epoch: [70] [  29/  40] time: 571.6337, loss: 0.00033415
Epoch: [70] [  30/  40] time: 571.7996, loss: 0.00000809
Epoch: [70] [  31/  40] time: 571.9574, loss: 0.00000993
Epoch: [70] [  32/  40] time: 572.1237, loss: 0.00056544
Epoch: [70] [  33/  40] time: 572.2935, loss: 0.00004953
Epoch: [70] [  34/  40] time: 572.4554, loss: 0.00020136
Epoch: [70] [  35/  40] time: 572.6086, loss: 0.00083397
Epoch: [70] [  36/  40] time: 572.7745, loss: 0.00015693
Epoch: [70] [  37/  40] time: 572.9273, loss: 0.00173849
Epoch: [70] [  38/  40] time: 573.1002, loss: 0.00089921
Epoch: [70] [  39/  40] time: 573.2611, loss: 0.00012050
[70/80] - ptime: 7.4653 loss: 0.00043922 acc: 0.70333 lr: 0.00047830
Epoch: [71] [   0/  40] time: 575.6520, loss: 0.00005464
Epoch: [71] [   1/  40] time: 575.8066, loss: 0.00869549
Epoch: [71] [   2/  40] time: 575.9726, loss: 0.00015723
Epoch: [71] [   3/  40] time: 576.1282, loss: 0.00000396
Epoch: [71] [   4/ 

Epoch: [74] [   8/  40] time: 601.4588, loss: 0.00011074
Epoch: [74] [   9/  40] time: 601.6150, loss: 0.00004085
Epoch: [74] [  10/  40] time: 601.7801, loss: 0.00004033
Epoch: [74] [  11/  40] time: 601.9339, loss: 0.00053335
Epoch: [74] [  12/  40] time: 602.1006, loss: 0.00000521
Epoch: [74] [  13/  40] time: 602.2567, loss: 0.00019531
Epoch: [74] [  14/  40] time: 602.4232, loss: 0.00000975
Epoch: [74] [  15/  40] time: 602.5857, loss: 0.00006159
Epoch: [74] [  16/  40] time: 602.7550, loss: 0.00003403
Epoch: [74] [  17/  40] time: 602.9071, loss: 0.00010019
Epoch: [74] [  18/  40] time: 603.0684, loss: 0.00009105
Epoch: [74] [  19/  40] time: 603.2254, loss: 0.00000763
Epoch: [74] [  20/  40] time: 603.3913, loss: 0.00003980
Epoch: [74] [  21/  40] time: 603.5481, loss: 0.00013562
Epoch: [74] [  22/  40] time: 603.7139, loss: 0.00008750
Epoch: [74] [  23/  40] time: 603.8744, loss: 0.00000790
Epoch: [74] [  24/  40] time: 604.0377, loss: 0.00079435
Epoch: [74] [  25/  40] time: 6

Epoch: [77] [  30/  40] time: 629.4885, loss: 0.00000287
Epoch: [77] [  31/  40] time: 629.6384, loss: 0.00293770
Epoch: [77] [  32/  40] time: 629.7970, loss: 0.00023823
Epoch: [77] [  33/  40] time: 629.9528, loss: 0.00016192
Epoch: [77] [  34/  40] time: 630.1161, loss: 0.00005714
Epoch: [77] [  35/  40] time: 630.2826, loss: 0.00110655
Epoch: [77] [  36/  40] time: 630.4528, loss: 0.00016939
Epoch: [77] [  37/  40] time: 630.6084, loss: 0.00038354
Epoch: [77] [  38/  40] time: 630.7705, loss: 0.00179823
Epoch: [77] [  39/  40] time: 630.9245, loss: 0.00011532
[77/80] - ptime: 7.3012 loss: 0.00130837 acc: 0.71000 lr: 0.00044429
Epoch: [78] [   0/  40] time: 632.7906, loss: 0.00006832
Epoch: [78] [   1/  40] time: 632.9479, loss: 0.00004286
Epoch: [78] [   2/  40] time: 633.1185, loss: 0.00000654
Epoch: [78] [   3/  40] time: 633.2821, loss: 0.00006243
Epoch: [78] [   4/  40] time: 633.4517, loss: 0.00004658
Epoch: [78] [   5/  40] time: 633.6081, loss: 0.00002813
Epoch: [78] [   6/ 

In [6]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放